In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
import re
from pprint import pprint
import country_converter as coco
import datetime
import pycountry

# Getting countries metadata from API

In [2]:
countries = 'https://api.covid19api.com/countries'

In [3]:
response_countries = requests.get(countries).json()
pprint(response_countries)

[{'Country': 'Belgium', 'ISO2': 'BE', 'Slug': 'belgium'},
 {'Country': 'Mexico', 'ISO2': 'MX', 'Slug': 'mexico'},
 {'Country': 'Israel', 'ISO2': 'IL', 'Slug': 'israel'},
 {'Country': 'Algeria', 'ISO2': 'DZ', 'Slug': 'algeria'},
 {'Country': 'Bosnia and Herzegovina',
  'ISO2': 'BA',
  'Slug': 'bosnia-and-herzegovina'},
 {'Country': 'Luxembourg', 'ISO2': 'LU', 'Slug': 'luxembourg'},
 {'Country': 'Samoa', 'ISO2': 'WS', 'Slug': 'samoa'},
 {'Country': 'Belize', 'ISO2': 'BZ', 'Slug': 'belize'},
 {'Country': 'Benin', 'ISO2': 'BJ', 'Slug': 'benin'},
 {'Country': 'Croatia', 'ISO2': 'HR', 'Slug': 'croatia'},
 {'Country': 'Lebanon', 'ISO2': 'LB', 'Slug': 'lebanon'},
 {'Country': 'Russian Federation', 'ISO2': 'RU', 'Slug': 'russia'},
 {'Country': 'South Africa', 'ISO2': 'ZA', 'Slug': 'south-africa'},
 {'Country': 'South Georgia and the South Sandwich Islands',
  'ISO2': 'GS',
  'Slug': 'south-georgia-and-the-south-sandwich-islands'},
 {'Country': 'Anguilla', 'ISO2': 'AI', 'Slug': 'anguilla'},
 {'C

In [ ]:
len(response_countries)

In [ ]:
name = []
slug = []
ISO2 = []
for i in range(len(response_countries)):
    name.append(response_countries[i]["Country"])
    slug.append(response_countries[i]["Slug"])
    ISO2.append(response_countries[i]["ISO2"])

countries_slug_df = pd.DataFrame({"Entity": name,
                                 "slug": slug,
                                 "ISO2": ISO2})
countries_slug_df.head()

In [ ]:
countries_slug_df.sort_values(by = ["Entity"], inplace = True, ignore_index = True)
countries_slug_df.head()

In [ ]:
countries_slug_df.to_csv("countries_raw.csv", index= False)

In [9]:
countries_slug_df = pd.read_csv("countries_raw.csv")
countries_slug_df = countries_slug_df[["Entity", "slug", "ISO2"]]
countries_slug_df.head()

,Entity,slug,ISO2
0,ALA Aland Islands,ala-aland-islands,AX
1,Afghanistan,afghanistan,AF
2,Albania,albania,AL
3,Algeria,algeria,DZ
4,American Samoa,american-samoa,AS


In [ ]:
len(countries_slug_df)

In [16]:
countries_slug_df.describe()

,Entity,slug,ISO2
count,248,248,247
unique,248,248,247
top,Dominica,tuvalu,TN
freq,1,1,1


# Standandizing country names for country metadata

In [10]:
entity1= list(set(countries_slug_df["Entity"]))
standard_names = coco.convert(names=entity1, to='name_short')
standardized_names1_df = pd.DataFrame({"Entity" : entity1,
                                      "standard_names": standard_names}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names1_df.head()

Netherlands Antilles not found in regex


,Entity,standard_names
0,ALA Aland Islands,Aland Islands
1,Afghanistan,Afghanistan
2,Albania,Albania
3,Algeria,Algeria
4,American Samoa,American Samoa


In [ ]:
std_countries_slug_df = pd.merge(countries_slug_df, standardized_names1_df, how = "left", left_on='Entity', right_on='Entity')
std_countries_slug_df.sort_values(by = "Entity", inplace = True)
std_countries_slug_df.head()

In [ ]:
std_countries_slug_df.to_csv("countries_slug.csv", index= False)

store response_countries in to a df.
Import csv into a df.
parse the country name column to add hyphens replacing spaces.
do a lookup on response_countries df to get the slugs.
create for loop to check all the slugs and make separate api calls.

# Importing stringency dataset

In [ ]:
stringency_df = pd.read_csv("covid-stringency-index.csv")
stringency_df.head()

In [ ]:
len(stringency_df)

In [ ]:
stringency_df["Entity"].nunique()

In [ ]:
stringency_df['month'] = pd.DatetimeIndex(stringency_df['Date']).month
stringency_df['year'] = pd.DatetimeIndex(stringency_df['Date']).year
stringency_df.head()

In [ ]:
stringency_df.loc[stringency_df['Entity'] == "Timor", 'Entity'] = "Timor-leste"

In [ ]:
stringency_df.describe()

In [ ]:
stringency_df.loc[stringency_df['Entity'] == "Timor-leste"]

In [ ]:
entity2= list(set(stringency_df["Entity"]))
standard_names2 = coco.convert(names=entity2, to='name_short')
standardized_names2_df = pd.DataFrame({"Entity" : entity2,
                                      "standard_names": standard_names2}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names2_df.head(10)

In [ ]:
cleaned_stringency_df = stringency_df.groupby(by = ["Entity", "Code", "month", "year"], as_index = False)["stringency_index"].mean()
cleaned_stringency_df.head(-10)

In [ ]:
std_stringency_df = pd.merge(cleaned_stringency_df, standardized_names2_df, how = "left", left_on='Entity', right_on='Entity')
std_stringency_df.sort_values(by = "Entity", inplace = True)
std_stringency_df.head()

In [ ]:
std_stringency_df.to_csv("cleaned_stringency.csv", index= False)

In [ ]:
merged_stringency_df =pd.merge(std_countries_slug_df, std_stringency_df, how = "inner", on = "standard_names")
merged_stringency_df = merged_stringency_df[merged_stringency_df["standard_names"] != "not found"]
merged_stringency_df.rename(columns = {"Entity_x": "Entity", "Code" : "ISO3"}, inplace=True)
merged_stringency_df = merged_stringency_df[["Entity" , "standard_names", "slug", "ISO2", "ISO3", "month", "year", "stringency_index"]]
merged_stringency_df.sort_values( by = ["standard_names", "year", "month"], inplace = True, ignore_index=True)
merged_stringency_df.head()

In [ ]:
merged_stringency_df.to_csv("merged_stringency.csv", index= False)

In [4]:
merged_stringency_df = pd.read_csv("merged_stringency.csv")

In [5]:
merged_stringency_df["standard_names"].nunique()

181

In [ ]:
std_countries_slug_df[["Entity"]][~std_countries_slug_df["standard_names"].isin(set(merged_stringency_df["standard_names"]))].drop_duplicates().reset_index(drop = True)

In [ ]:
std_stringency_df[["Entity"]][~std_stringency_df["standard_names"].isin(list(set(merged_stringency_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

# Getting coordinates of countries

In [ ]:
country_names= list(set(merged_stringency_df['slug']))
country_names.sort()
country_names

In [ ]:
len(country_names)

In [ ]:
country_std_names= list(set(merged_stringency_df['standard_names']))
country_std_names.sort()
country_std_names

In [ ]:
# Target country
# target_country = "India"
lat = []
lng = []
contry_geo_name = []
# Build the endpoint URL
for country in range(len(country_std_names)):
    geo_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(country_std_names[country], gkey)
    geo_data = requests.get(geo_url).json()
    contry_geo_name.append(geo_data["results"][0]["address_components"][0]["long_name"])
    lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
    time.sleep(1)
# pprint(geo_data)

In [ ]:
contry_coord = pd.DataFrame({"Entity": contry_geo_name,
                            "lat": lat,
                            "lng" : lng,
                            "standard_names" : country_std_names})
contry_coord.head()

In [ ]:
contry_coord.to_csv("contry_coord.csv", index= False)

In [ ]:
contry_coord = pd.read_csv("contry_coord.csv")

In [ ]:
contry_coord["Entity"].nunique()

# Making API calls to get covid stats for each country

In [ ]:
base_url = 'https://api.covid19api.com/total/country/'
# https://api.covid19api.com/total/country/india/status/confirmed
entity = []
confirmed = []
deaths = []
recovered = []
active = []
date = []

In [ ]:
# test_url = "https://api.covid19api.com/total/country/albania"
# covid_response = requests.get(test_url).json()
# print(len(covid_response))

In [ ]:
for country in range(len(country_names)):
    search_url = base_url + country_names[country]
    covid_response = requests.get(search_url).json()
#     print(len(covid_response))
    for i in range(len(covid_response)):
        entity.append(covid_response[i]["Country"])
        confirmed.append(covid_response[i]["Confirmed"])
        deaths.append(covid_response[i]["Deaths"])
        recovered.append(covid_response[i]["Recovered"])
        active.append(covid_response[i]["Active"])
        date.append(covid_response[i]["Date"])
    time.sleep(1)

## Storing the API responses in a dataframe

In [ ]:
global_covid_df = pd.DataFrame({"Entity" : entity,
                               "Confirmed" : confirmed,
                               "Deaths" : deaths,
                               "Recovered" : recovered,
                               "Active" : active,
                               "Date" : date})
global_covid_df.sort_values(by = ["Entity", "Date"], inplace = True, ignore_index= True)
global_covid_df.head(-10)

In [ ]:
search_url = "https://api.covid19api.com/total/country/india"
covid_response = requests.get(search_url).json()
covid_response[0]["Date"]

In [ ]:
global_covid_df.to_csv("global_covid_stats.csv", index = False)

In [4]:
global_covid_df = pd.read_csv("global_covid_stats.csv")

global_covid_df.head()

,Entity,Confirmed,Deaths,Recovered,Active,Date
0,Afghanistan,0,0,0,0,2020-01-22T00:00:00Z
1,Afghanistan,0,0,0,0,2020-01-23T00:00:00Z
2,Afghanistan,0,0,0,0,2020-01-24T00:00:00Z
3,Afghanistan,0,0,0,0,2020-01-25T00:00:00Z
4,Afghanistan,0,0,0,0,2020-01-26T00:00:00Z


### Breaking the cumulative counts into daily counts

In [5]:
global_covid_df["Confirmed_daily"] =global_covid_df["Confirmed"].diff().fillna(0)
global_covid_df["Deaths_daily"] =global_covid_df["Deaths"].diff().fillna(0)
global_covid_df["Recovered_daily"] =global_covid_df["Recovered"].diff().fillna(0)
global_covid_df["Active_daily"] =global_covid_df["Active"].diff().fillna(0)

# num = global_covid_df._get_numeric_data()
# num[num < 0] = 0
global_covid_df.loc[global_covid_df['Confirmed_daily'] < 0 , 'Confirmed_daily'] = global_covid_df["Confirmed"]
global_covid_df.loc[global_covid_df['Deaths_daily'] < 0 , 'Deaths_daily'] = global_covid_df["Deaths"]
global_covid_df.loc[global_covid_df['Recovered_daily'] < 0 , 'Recovered_daily'] = global_covid_df["Recovered"]
global_covid_df.loc[global_covid_df['Active_daily'] < 0 , 'Active_daily'] = global_covid_df["Active"]
global_covid_df = global_covid_df[['Entity','Date', 'Confirmed_daily', 'Deaths_daily', 'Recovered_daily',
       'Active_daily']]
global_covid_df.head(-10)

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0
1,Afghanistan,2020-01-23T00:00:00Z,0.0,0.0,0.0,0.0
2,Afghanistan,2020-01-24T00:00:00Z,0.0,0.0,0.0,0.0
3,Afghanistan,2020-01-25T00:00:00Z,0.0,0.0,0.0,0.0
4,Afghanistan,2020-01-26T00:00:00Z,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
57044,Zimbabwe,2020-12-04T00:00:00Z,123.0,1.0,48.0,74.0
57045,Zimbabwe,2020-12-05T00:00:00Z,70.0,10.0,42.0,18.0
57046,Zimbabwe,2020-12-06T00:00:00Z,101.0,0.0,36.0,65.0
57047,Zimbabwe,2020-12-07T00:00:00Z,121.0,3.0,92.0,26.0


In [ ]:
global_covid_df["Entity"].nunique()

In [ ]:
len(global_covid_df)

In [ ]:
# global_covid_df.to_csv("global_covid_daily.csv")

In [6]:
global_covid_df['month'] = pd.DatetimeIndex(global_covid_df['Date']).month
global_covid_df['year'] = pd.DatetimeIndex(global_covid_df['Date']).year
global_covid_df.head()

,Entity,Date,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,month,year
0,Afghanistan,2020-01-22T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
1,Afghanistan,2020-01-23T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
2,Afghanistan,2020-01-24T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
3,Afghanistan,2020-01-25T00:00:00Z,0.0,0.0,0.0,0.0,1,2020
4,Afghanistan,2020-01-26T00:00:00Z,0.0,0.0,0.0,0.0,1,2020


### Get monthly stats per country

In [7]:
global_covid_monthly_df = global_covid_df.groupby(["Entity", "month", "year"], as_index = False).agg({"Confirmed_daily": "sum",
                                                                                   "Deaths_daily": "sum",
                                                                                   "Recovered_daily": "sum",
                                                                                   "Active_daily" : "sum"})
global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0
...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,6702.0
2049,Zambia,11,2020,1215.0,8.0,1318.0,7226.0
2050,Zambia,12,2020,928.0,16.0,731.0,1781.0
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0


In [10]:
len(global_covid_monthly_df)

2063

In [11]:
std_global_covid_monthly_df = pd.merge(global_covid_monthly_df,standardized_names1_df, how = "inner", on = "Entity")
std_global_covid_monthly_df.head(-10)

,Entity,month,year,Confirmed_daily,Deaths_daily,Recovered_daily,Active_daily,standard_names
0,Afghanistan,1,2020,0.0,0.0,0.0,0.0,Afghanistan
1,Afghanistan,2,2020,1.0,0.0,0.0,1.0,Afghanistan
2,Afghanistan,3,2020,174.0,4.0,5.0,165.0,Afghanistan
3,Afghanistan,4,2020,1952.0,60.0,255.0,1637.0,Afghanistan
4,Afghanistan,5,2020,13081.0,194.0,1068.0,11819.0,Afghanistan
...,...,...,...,...,...,...,...,...
2048,Zambia,10,2020,1673.0,17.0,1721.0,6702.0,Zambia
2049,Zambia,11,2020,1215.0,8.0,1318.0,7226.0,Zambia
2050,Zambia,12,2020,928.0,16.0,731.0,1781.0,Zambia
2051,Zimbabwe,1,2020,0.0,0.0,0.0,0.0,Zimbabwe


In [ ]:
std_global_covid_monthly_df.loc[std_global_covid_monthly_df["standard_names"] == "Macau"]

In [ ]:
std_global_covid_monthly_df["standard_names"].nunique()

In [ ]:
std_global_covid_monthly_stringency_df = pd.merge(std_global_covid_monthly_df, merged_stringency_df , 
                                                  how = "inner", on = ["standard_names", "month", "year"])
std_global_covid_monthly_stringency_df = std_global_covid_monthly_stringency_df[['standard_names', 'month', 'year',
        'Confirmed_daily', 'Deaths_daily', 'Recovered_daily', 'Active_daily', 'stringency_index']]
std_global_covid_monthly_stringency_df.head(-10)

In [ ]:
std_global_covid_monthly_stringency_df["standard_names"].nunique()

In [ ]:
main_df = pd.merge(std_global_covid_monthly_stringency_df, contry_coord, how = "left", on = "standard_names")
main_df = main_df[['standard_names', 'month', 'year', 'Confirmed_daily', 'Deaths_daily',
       'Recovered_daily', 'Active_daily', 'stringency_index', 'lat','lng']]
main_df.rename(columns = {'Confirmed_daily' : "Confirmed", 'Deaths_daily' : "Deaths",
                           'Recovered_daily': "Recovered", 'Active_daily' : "Active"}, inplace = True)
main_df.head(-10)

In [ ]:
main_df["standard_names"].nunique()

In [ ]:
main_df.loc[~main_df["standard_names"].isin(list(set(merged_stringency_df["standard_names"])))]\
                        .drop_duplicates().reset_index(drop = True)

In [ ]:
main_df.loc[~main_df["standard_names"].isin(list(set(contry_coord["standard_names"])))]\
                        .drop_duplicates().reset_index(drop = True)

In [ ]:
main_df.to_csv("main.csv", index= False)

In [7]:
main_df = pd.read_csv("main.csv")

In [13]:
main_stats_df = main_df.groupby("standard_names", as_index = False).agg({"Confirmed": "sum",
                                                                                   "Deaths": "sum",
                                                                                   "Recovered": "sum",
                                                                                   "Active" : "sum"})
main_stats_df.sort_values('Confirmed', inplace= True, ascending=False, ignore_index=True)

main_stats_df.head(10)

,standard_names,Confirmed,Deaths,Recovered,Active
0,United States,17459296.0,313588.0,11060515.0,25347151.0
1,India,9462809.0,137626.0,8889585.0,47550412.0
2,Brazil,7162978.0,185650.0,6322955.0,37094627.0
3,China,6665209.0,14051.0,166939.0,7616036.0
4,France,5352449.0,338941.0,375431.0,7072707.0
5,Russia,2764843.0,49170.0,2207398.0,20563454.0
6,Spain,2246032.0,106259.0,150376.0,3547225.0
7,United Kingdom,2173896.0,1625676.0,8295.0,6503966.0
8,Italy,2159937.0,102569.0,1434624.0,19787560.0
9,Turkey,1982090.0,17610.0,1753552.0,5133345.0


In [14]:
all_zero_df = main_stats_df.loc[main_stats_df["Confirmed"] == 0]
all_zero_df

,standard_names,Confirmed,Deaths,Recovered,Active
171,Macau,0.0,0.0,0.0,0.0


In [15]:
main_df_dropped = main_df.loc[~main_df["standard_names"].isin(list(set(all_zero_df["standard_names"])))]\
                        .drop_duplicates().reset_index(drop = True)
main_df_dropped["standard_names"].nunique()

171

In [16]:
main_stats_df.sort_values('Confirmed', ascending=True, ignore_index=True)

,standard_names,Confirmed,Deaths,Recovered,Active
0,Macau,0.0,0.0,0.0,0.0
1,Vanuatu,1.0,0.0,1.0,1.0
2,Solomon Islands,17.0,0.0,5.0,16.0
3,Timor-Leste,31.0,0.0,30.0,63.0
4,Laos,41.0,0.0,46.0,215.0
...,...,...,...,...,...
167,France,5352449.0,338941.0,375431.0,7072707.0
168,China,6665209.0,14051.0,166939.0,7616036.0
169,Brazil,7162978.0,185650.0,6322955.0,37094627.0
170,India,9462809.0,137626.0,8889585.0,47550412.0


In [ ]:
main_stats_df.to_csv("main_stats_2020.csv", index= False)

# Import population dataset

In [4]:
popluation_2020 = pd.read_csv("population_by_country_2020.csv")
popluation_2020.head(-10)

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
220,Caribbean Netherlands,26265,0.94 %,244,80,328,NaN,N.A.,N.A.,75 %,0.00 %
221,Palau,18109,0.48 %,86,39,460,NaN,N.A.,N.A.,N.A.,0.00 %
222,Cook Islands,17567,0.09 %,16,73,240,NaN,N.A.,N.A.,75 %,0.00 %
223,Anguilla,15026,0.90 %,134,167,90,NaN,N.A.,N.A.,N.A.,0.00 %


## Get 10 top, 10 countries around mean, 10 countries with least numbers

In [5]:
popluation_2020["Country (or dependency)"].nunique()

235

In [8]:
popluation_2020[["Country (or dependency)"]][~popluation_2020["Country (or dependency)"].isin(list(set(main_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

,Country (or dependency)
0,Côte d'Ivoire
1,North Korea
2,Czech Republic (Czechia)
3,Hong Kong
4,Kyrgyzstan
...,...
66,Montserrat
67,Falkland Islands
68,Niue
69,Tokelau


In [9]:
entity3= list(set(popluation_2020["Country (or dependency)"]))
standard_names3 = coco.convert(names=entity3, to='name_short')
standardized_names3_df = pd.DataFrame({"Country (or dependency)" : entity3,
                                      "standard_names": standard_names3}).sort_values(by = "Country (or dependency)").reset_index(drop = True)
standardized_names3_df.head(10)

Channel Islands not found in regex


,Country (or dependency),standard_names
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,American Samoa,American Samoa
4,Andorra,Andorra
5,Angola,Angola
6,Anguilla,Anguilla
7,Antigua and Barbuda,Antigua and Barbuda
8,Argentina,Argentina
9,Armenia,Armenia


In [19]:
std_population_df = pd.merge(popluation_2020, standardized_names3_df, how = "left", on = "Country (or dependency)")
std_population_df = std_population_df[["standard_names", "Population (2020)", "Density (P/Km²)"]].dropna()
std_population_df.head()

,standard_names,Population (2020),Density (P/Km²)
0,China,1440297825,153
1,India,1382345085,464
2,United States,331341050,36
3,Indonesia,274021604,151
4,Pakistan,221612785,287


In [11]:
std_population_df["standard_names"].nunique()

235

In [20]:
ultimate_df = pd.merge(main_df_dropped, std_population_df, how = "inner", on = "standard_names")
ultimate_df["Infection_rate"] = ((ultimate_df["Confirmed"]/ ultimate_df["Population (2020)"]*100).map("{:.4f}%".format))
ultimate_df = ultimate_df[["standard_names", "month", "year", "Confirmed", "stringency_index"
                          , "lat", "lng", "Population (2020)", "Density (P/Km²)", "Infection_rate" ]]
ultimate_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Density (P/Km²),Infection_rate
0,Afghanistan,1,2020,0.0,0.000000,33.93911,67.709953,39074280,60,0.0000%
1,Afghanistan,2,2020,1.0,2.010690,33.93911,67.709953,39074280,60,0.0000%
2,Afghanistan,3,2020,174.0,40.831935,33.93911,67.709953,39074280,60,0.0004%
3,Afghanistan,4,2020,1952.0,80.740000,33.93911,67.709953,39074280,60,0.0050%
4,Afghanistan,5,2020,13081.0,84.260000,33.93911,67.709953,39074280,60,0.0335%


In [ ]:
ultimate_df["standard_names"].nunique()

In [ ]:
main_df[["standard_names"]][~main_df["standard_names"].isin(list(set(ultimate_df["standard_names"])))].drop_duplicates().reset_index(drop = True)

In [ ]:
ultimate_df.head(20)

In [21]:
ultimate_df.to_csv("ultimate.csv", index= False)

In [4]:
ultimate_df = pd.read_csv("ultimate.csv")
ultimate_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate
0,Afghanistan,1,2020,0,0.000000,33.93911,67.709953,39074280,0.00%
1,Afghanistan,2,2020,1,2.010690,33.93911,67.709953,39074280,0.00%
2,Afghanistan,3,2020,174,40.831935,33.93911,67.709953,39074280,0.00%
3,Afghanistan,4,2020,1952,80.740000,33.93911,67.709953,39074280,0.01%
4,Afghanistan,5,2020,13081,84.260000,33.93911,67.709953,39074280,0.03%


## Getting ISO2 and ISO3 codes for all countries

In [25]:
country_name = []
ISO_2 = []
ISO_3 = []
for country in pycountry.countries:
    country_name.append(country.name)
    ISO_2.append(country.alpha_2)
    ISO_3.append(country.alpha_3)
    
country_ISO_df = pd.DataFrame({"Entity" : country_name,
                               "ISO_2" : ISO_2,
                               "ISO_3" : ISO_3})
country_ISO_df.head()

,Entity,ISO_2,ISO_3
0,Aruba,AW,ABW
1,Afghanistan,AF,AFG
2,Angola,AO,AGO
3,Anguilla,AI,AIA
4,Åland Islands,AX,ALA


In [26]:
entity3= list(set(country_ISO_df["Entity"]))
standard_names = coco.convert(names=entity3, to='name_short')
standardized_names4_df = pd.DataFrame({"Entity" : entity3,
                                      "standard_names": standard_names}).sort_values(by = "Entity").reset_index(drop = True)
standardized_names4_df.head()

,Entity,standard_names
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,American Samoa,American Samoa
4,Andorra,Andorra


In [27]:
country_ISO_df = pd.merge(country_ISO_df, standardized_names4_df, how = "inner", on = "Entity")
country_ISO_df = country_ISO_df[["standard_names", "ISO_2", "ISO_3"]]
country_ISO_df.head()

,standard_names,ISO_2,ISO_3
0,Aruba,AW,ABW
1,Afghanistan,AF,AFG
2,Angola,AO,AGO
3,Anguilla,AI,AIA
4,Aland Islands,AX,ALA


In [28]:
country_ISO_df["standard_names"].nunique()

249

# Creating the final dataset for plotting

In [29]:
ultimate_df = pd.merge(ultimate_df, country_ISO_df, how = "inner" , on = "standard_names")
ultimate_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Density (P/Km²),Infection_rate,ISO_2,ISO_3
0,Afghanistan,1,2020,0.0,0.000000,33.93911,67.709953,39074280,60,0.0000%,AF,AFG
1,Afghanistan,2,2020,1.0,2.010690,33.93911,67.709953,39074280,60,0.0000%,AF,AFG
2,Afghanistan,3,2020,174.0,40.831935,33.93911,67.709953,39074280,60,0.0004%,AF,AFG
3,Afghanistan,4,2020,1952.0,80.740000,33.93911,67.709953,39074280,60,0.0050%,AF,AFG
4,Afghanistan,5,2020,13081.0,84.260000,33.93911,67.709953,39074280,60,0.0335%,AF,AFG


In [10]:
ultimate_df["standard_names"].nunique()

170

In [30]:
ultimate_df.to_csv("ultimate.csv", index = False)

In [37]:
ultimate_nan_df = ultimate_df[ultimate_df["stringency_index"].isnull()]
ultimate_nan_df

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3


In [38]:
ultimate_nan_df.to_csv("ultimate_nan.csv", index=False)

In [37]:
ultimate_no_null_df = ultimate_df.dropna()
ultimate_no_null_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Density (P/Km²),Infection_rate,ISO_2,ISO_3
0,Afghanistan,1,2020,0.0,0.000000,33.93911,67.709953,39074280,60,0.0000%,AF,AFG
1,Afghanistan,2,2020,1.0,2.010690,33.93911,67.709953,39074280,60,0.0000%,AF,AFG
2,Afghanistan,3,2020,174.0,40.831935,33.93911,67.709953,39074280,60,0.0004%,AF,AFG
3,Afghanistan,4,2020,1952.0,80.740000,33.93911,67.709953,39074280,60,0.0050%,AF,AFG
4,Afghanistan,5,2020,13081.0,84.260000,33.93911,67.709953,39074280,60,0.0335%,AF,AFG


In [38]:
ultimate_no_null_df.to_csv("ultimate_no_null.csv" , index=False)

In [2]:
ultimate_no_null_df= pd.read_csv("ultimate_no_null.csv")
ultimate_no_null_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Density (P/Km²),Infection_rate,ISO_2,ISO_3,Infection_rate_f
0,Afghanistan,1,2020,0.0,0.000000,33.93911,67.709953,39074280,60,0.0000%,AF,AFG,0.0000
1,Afghanistan,2,2020,1.0,2.010690,33.93911,67.709953,39074280,60,0.0000%,AF,AFG,0.0000
2,Afghanistan,3,2020,174.0,40.831935,33.93911,67.709953,39074280,60,0.0004%,AF,AFG,0.0004
3,Afghanistan,4,2020,1952.0,80.740000,33.93911,67.709953,39074280,60,0.0050%,AF,AFG,0.0050
4,Afghanistan,5,2020,13081.0,84.260000,33.93911,67.709953,39074280,60,0.0335%,AF,AFG,0.0335


In [14]:
ultimate_df["standard_names"].nunique()

170

In [15]:
ultimate_no_null_df["standard_names"].nunique()

170

In [41]:
ultimate_no_null_df["Infection_rate_f"] = ultimate_no_null_df["Infection_rate"].str.rstrip('%').astype('float')

In [42]:
ultimate_no_null_df.head()

,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Density (P/Km²),Infection_rate,ISO_2,ISO_3,Infection_rate_f
0,Afghanistan,1,2020,0.0,0.000000,33.93911,67.709953,39074280,60,0.0000%,AF,AFG,0.0000
1,Afghanistan,2,2020,1.0,2.010690,33.93911,67.709953,39074280,60,0.0000%,AF,AFG,0.0000
2,Afghanistan,3,2020,174.0,40.831935,33.93911,67.709953,39074280,60,0.0004%,AF,AFG,0.0004
3,Afghanistan,4,2020,1952.0,80.740000,33.93911,67.709953,39074280,60,0.0050%,AF,AFG,0.0050
4,Afghanistan,5,2020,13081.0,84.260000,33.93911,67.709953,39074280,60,0.0335%,AF,AFG,0.0335


In [43]:
ultimate_no_null_df.to_csv("ultimate_no_null.csv" , index=False)

# Sample dataframe creation

In [3]:
covid_stats_df = ultimate_no_null_df.groupby(["standard_names", "ISO_2", "ISO_3", "Population (2020)", "lat", "lng", "Density (P/Km²)"], as_index= False).agg({"Confirmed" : "sum",
                                                                                      "stringency_index": "mean",
                                                                                      "Infection_rate_f" : "sum"})
covid_stats_df.to_csv("covid_stats_2020.csv", index=False)
covid_stats_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,60,46274.0,42.700514,0.1183
1,Albania,AL,ALB,2877239,41.153332,20.168331,105,52004.0,55.330056,1.8074
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,18,94371.0,59.981444,0.2146
3,Andorra,AD,AND,77287,42.506285,1.521801,164,7519.0,40.286864,9.7286
4,Angola,AO,AGO,33032075,-11.202692,17.873887,26,16562.0,57.541104,0.0502


In [4]:
covid_stats_df =  pd.read_csv("covid_stats_2020.csv")
covid_stats_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,60,46274.0,42.700514,0.1183
1,Albania,AL,ALB,2877239,41.153332,20.168331,105,52004.0,55.330056,1.8074
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,18,94371.0,59.981444,0.2146
3,Andorra,AD,AND,77287,42.506285,1.521801,164,7519.0,40.286864,9.7286
4,Angola,AO,AGO,33032075,-11.202692,17.873887,26,16562.0,57.541104,0.0502


In [5]:
covid_stats_sorted_df = covid_stats_df.sort_values(by = "Confirmed", ascending = False, ignore_index= True)
covid_stats_sorted_df.to_csv("covid_stats_sorted.csv", index= False)
covid_stats_sorted_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
0,United States,US,USA,331341050,37.090240,-95.712891,36,17459296.0,56.045880,5.2692
1,India,IN,IND,1382345085,20.593684,78.962880,464,9462809.0,62.869497,0.6845
2,Brazil,BR,BRA,212821986,-14.235004,-51.925280,25,7162978.0,56.761687,3.3657
3,China,CN,CHN,1440297825,35.861660,104.195397,153,6665209.0,68.391841,0.4628
4,France,FR,FRA,65298930,46.227638,2.213749,119,5352449.0,55.223795,8.1966


In [6]:
covid_stats_sorted_df_top = covid_stats_sorted_df.head(10)
covid_stats_sorted_df_top

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
0,United States,US,USA,331341050,37.090240,-95.712891,36,17459296.0,56.045880,5.2692
1,India,IN,IND,1382345085,20.593684,78.962880,464,9462809.0,62.869497,0.6845
2,Brazil,BR,BRA,212821986,-14.235004,-51.925280,25,7162978.0,56.761687,3.3657
3,China,CN,CHN,1440297825,35.861660,104.195397,153,6665209.0,68.391841,0.4628
4,France,FR,FRA,65298930,46.227638,2.213749,119,5352449.0,55.223795,8.1966
5,Russia,RU,RUS,145945524,61.524010,105.318756,9,2764843.0,50.600998,1.8944
6,Spain,ES,ESP,46757980,40.463667,-3.749220,94,2246032.0,55.975557,4.8034
7,United Kingdom,GB,GBR,67948282,55.378051,-3.435973,281,2173896.0,56.888553,3.1993
8,Italy,IT,ITA,60446035,41.871940,12.567380,206,2159937.0,58.364239,3.5734
9,Turkey,TR,TUR,84495243,38.963745,35.243322,110,1982090.0,53.585087,2.3457


In [8]:
covid_stats_sorted_df_bottom = covid_stats_sorted_df.tail(10)
covid_stats_sorted_df_bottom

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
159,Barbados,BB,BRB,287437,13.193887,-59.543198,668,307.0,45.059213,0.1069
160,Seychelles,SC,SYC,98453,-4.679574,55.491977,214,202.0,33.860586,0.2051
161,Monaco,MC,MCO,39290,43.738418,7.424616,26337,154.0,54.186199,0.3919
162,Brunei Darussalam,BN,BRN,438202,4.535277,114.727669,83,152.0,38.282705,0.0349
163,Dominica,DM,DMA,72017,15.414999,-61.370976,96,88.0,35.822549,0.1224
164,Fiji,FJ,FJI,897573,-17.713371,178.065032,49,46.0,51.240738,0.0050
165,Laos,LA,LAO,7293542,19.856270,102.495496,32,41.0,34.867858,0.0004
166,Timor-Leste,TL,TLS,1322667,-8.874217,125.727539,89,31.0,31.385059,0.0024
167,Solomon Islands,SB,SLB,689671,-9.645710,160.156194,25,17.0,31.102298,0.0025
168,Vanuatu,VU,VUT,308337,-15.376706,166.959158,25,1.0,38.589624,0.0003


In [9]:
def get_median_index(d):
    ranks = d.rank(pct=True)
    close_to_median = abs(ranks - 0.5)
    return close_to_median.idxmin()
median_index = list(covid_stats_sorted_df[["Confirmed"]].apply(get_median_index, 0))
# type(median_index)
median_index[0]

84

In [10]:
covid_stats_sorted_df_mid = covid_stats_sorted_df.iloc[ median_index[0]-5:median_index[0]+5]
covid_stats_sorted_df_mid

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
79,Ireland,IE,IRL,4947267,53.142367,-7.692054,72,78254.0,56.331917,1.5817
80,Nigeria,NG,NGA,206984347,9.081999,8.675277,226,77013.0,53.885979,0.0372
81,Uzbekistan,UZ,UZB,33551824,41.377491,64.585262,79,75675.0,50.968375,0.2255
82,Malaysia,MY,MYS,32436963,4.210484,101.975766,99,65697.0,52.726719,0.2026
83,Singapore,SG,SGP,5858322,1.352083,103.819836,8358,58386.0,49.634418,0.9967
84,Ghana,GH,GHA,31181428,7.946527,-1.023194,137,53653.0,41.623758,0.1721
85,Albania,AL,ALB,2877239,41.153332,20.168331,105,52004.0,55.330056,1.8074
86,Luxembourg,LU,LUX,627704,49.815273,6.129583,242,51656.0,43.160278,8.2295
87,South Korea,KR,KOR,51276977,35.907757,127.766922,527,48570.0,47.322044,0.0945
88,Finland,FI,FIN,5542237,61.924110,25.748151,18,47179.0,36.915609,0.8511


In [11]:
sample = covid_stats_sorted_df_top.append(covid_stats_sorted_df_mid, ignore_index = True)
sample = sample.append(covid_stats_sorted_df_bottom, ignore_index = True)
sample

,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Density (P/Km²),Confirmed,stringency_index,Infection_rate_f
0,United States,US,USA,331341050,37.090240,-95.712891,36,17459296.0,56.045880,5.2692
1,India,IN,IND,1382345085,20.593684,78.962880,464,9462809.0,62.869497,0.6845
2,Brazil,BR,BRA,212821986,-14.235004,-51.925280,25,7162978.0,56.761687,3.3657
3,China,CN,CHN,1440297825,35.861660,104.195397,153,6665209.0,68.391841,0.4628
4,France,FR,FRA,65298930,46.227638,2.213749,119,5352449.0,55.223795,8.1966
5,Russia,RU,RUS,145945524,61.524010,105.318756,9,2764843.0,50.600998,1.8944
6,Spain,ES,ESP,46757980,40.463667,-3.749220,94,2246032.0,55.975557,4.8034
7,United Kingdom,GB,GBR,67948282,55.378051,-3.435973,281,2173896.0,56.888553,3.1993
8,Italy,IT,ITA,60446035,41.871940,12.567380,206,2159937.0,58.364239,3.5734
9,Turkey,TR,TUR,84495243,38.963745,35.243322,110,1982090.0,53.585087,2.3457


In [12]:
sample.to_csv("samples.csv", index= False)

# test dataframe creation

In [13]:
test1_df = ultimate_no_null_df[["standard_names",'ISO_2', 'ISO_3', 'Population (2020)', 'Density (P/Km²)',"month", "Confirmed", "Infection_rate_f"]].loc[ultimate_no_null_df.groupby("standard_names")["Confirmed"].idxmax()].reset_index(drop= True)
test1_df.head()

,standard_names,ISO_2,ISO_3,Population (2020),Density (P/Km²),month,Confirmed,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,60,6,16299.0,0.0417
1,Albania,AL,ALB,2877239,105,11,17307.0,0.6015
2,Algeria,DZ,DZA,43984569,18,11,25257.0,0.0574
3,Andorra,AD,AND,77287,164,10,2706.0,3.5012
4,Angola,AO,AGO,33032075,26,10,5833.0,0.0177


In [14]:
test2_df = ultimate_no_null_df.groupby("standard_names", as_index = False).agg(total_inf_rate = ("Infection_rate_f", "sum"))
test2_df.head()

,standard_names,total_inf_rate
0,Afghanistan,0.1183
1,Albania,1.8074
2,Algeria,0.2146
3,Andorra,9.7286
4,Angola,0.0502


In [15]:
strngncy_df = ultimate_no_null_df[["standard_names", "stringency_index"]].loc[(ultimate_no_null_df["month"] <=3) ].reset_index(drop = True)
strngncy_df.head(15)

,standard_names,stringency_index
0,Afghanistan,0.000000
1,Afghanistan,2.010690
2,Afghanistan,40.831935
3,Albania,0.000000
4,Albania,3.545172
5,Albania,58.570968
6,Algeria,0.000000
7,Algeria,0.000000
8,Algeria,37.486129
9,Andorra,0.000000


In [16]:
strngncy_df['stringency_hike_1Q'] = strngncy_df.groupby('standard_names')['stringency_index'].transform(lambda x: x.iat[-1] - x.iat[0])
test3_df = strngncy_df[["standard_names", "stringency_hike_1Q"]].drop_duplicates().reset_index(drop = True)
test3_df.head()

,standard_names,stringency_hike_1Q
0,Afghanistan,40.831935
1,Albania,58.570968
2,Algeria,37.486129
3,Andorra,21.624839
4,Angola,24.400000


In [17]:
test_df = pd.merge(test1_df, test2_df, how = "left", on = "standard_names")
test_df = pd.merge(test_df, test3_df, how = "left", on = "standard_names") 
test_df

,standard_names,ISO_2,ISO_3,Population (2020),Density (P/Km²),month,Confirmed,Infection_rate_f,total_inf_rate,stringency_hike_1Q
0,Afghanistan,AF,AFG,39074280,60,6,16299.0,0.0417,0.1183,40.831935
1,Albania,AL,ALB,2877239,105,11,17307.0,0.6015,1.8074,58.570968
2,Algeria,DZ,DZA,43984569,18,11,25257.0,0.0574,0.2146,37.486129
3,Andorra,AD,AND,77287,164,10,2706.0,3.5012,9.7286,21.624839
4,Angola,AO,AGO,33032075,26,10,5833.0,0.0177,0.0502,24.400000
...,...,...,...,...,...,...,...,...,...,...
165,Venezuela,VE,VEN,28421581,32,9,28394.0,0.0999,0.3848,52.150968
166,Vietnam,VN,VNM,97490013,314,8,486.0,0.0005,0.0016,50.714516
167,Yemen,YE,YEM,29935468,56,8,2062.0,0.0069,0.0135,18.458710
168,Zambia,ZM,ZMB,18468257,25,8,6134.0,0.0332,0.1006,24.016774


In [18]:
test_df.to_csv("test_df.csv", index = False)